In [1]:
import numpy as np                      # Sayısal işlemler ve matematiksel hesaplamalar için NumPy
import pandas as pd                     # Veri tablosu (DataFrame) oluşturmak için Pandas

from sklearn.neural_network import MLPRegressor      # Yapay Sinir Ağı (ANN) regresyon modeli
from sklearn.preprocessing import StandardScaler     # Verileri standartlaştırmak için
from sklearn.model_selection import LeaveOneOut      # Leave-One-Out çapraz doğrulama yöntemi
from sklearn.metrics import mean_absolute_error, mean_squared_error  # Performans metrikleri

# =========================
# 1) DATASET (Bölüm 0'dan)
# =========================
data = {
    "Engel_Sayisi": [41, 34, 30, 41, 16, 44, 20, 49],   # Her senaryodaki engel sayıları
    "epsilon":      [0.9023, 1.6423, 0.3634, 0.8725, 1.6244, 1.5656, 1.2872, 0.4353],  # epsilon değerleri
    "kappa":        [3.5984, 8.7080, 9.7813, 7.0718, 1.6087, 8.7852, 1.6190, 3.3939],  # kappa değerleri
    "Sapma_A":      [2.8514, 6.2783, 7.6724, 5.2941, 1.3108, 6.3413, 1.3663, 3.0648],  # Sapma A değerleri
    "Sapma_B":      [1.6610, 3.6050, 4.4628, 3.0579, 0.7666, 3.6418, 0.8030, 1.8170]   # Sapma B değerleri
}

df = pd.DataFrame(data)                # Sözlük yapısı Pandas DataFrame’e dönüştürülür

# =========================
# 2) Hedef değişken
# =========================
df["Zorluk"] = (df["Sapma_A"] + df["Sapma_B"]) / 2.0  # Sapma A ve B’nin ortalaması alınarak Zorluk hesaplanır

X = df[["Engel_Sayisi", "epsilon", "kappa"]].values  # Giriş (bağımsız) değişkenler belirlenir
y = df["Zorluk"].values                              # Çıkış (hedef) değişken belirlenir

# =========================
# 3) Standardizasyon
# =========================
scaler = StandardScaler()            # StandardScaler nesnesi oluşturulur
X_scaled = scaler.fit_transform(X)   # Giriş verileri ölçeklendirilir (ortalama=0, std=1)

# =========================
# 4) ANN Modeli
# =========================
model = MLPRegressor(
    hidden_layer_sizes=(16, 8),   # İki gizli katman: ilkinde 16, ikincide 8 nöron
    activation="relu",            # Aktivasyon fonksiyonu olarak ReLU kullanılır
    solver="adam",                # Ağırlık güncellemesi için Adam algoritması kullanılır
    max_iter=2000,                # Maksimum iterasyon (eğitim adımı) sayısı
    random_state=42               # Sonuçların tekrarlanabilir olması için sabit seed
)

# =========================
# 5) Leave-One-Out CV
# =========================
loo = LeaveOneOut()     # Leave-One-Out çapraz doğrulama yöntemi tanımlanır

y_true = []             # Gerçek zorluk değerlerini saklamak için liste
y_pred = []             # Tahmin edilen zorluk değerlerini saklamak için liste

for train_index, test_index in loo.split(X_scaled):   # Her seferinde 1 veri test, kalanlar eğitim
    X_train, X_test = X_scaled[train_index], X_scaled[test_index]  # Eğitim ve test giriş verileri
    y_train, y_test = y[train_index], y[test_index]                # Eğitim ve test hedef verileri

    model.fit(X_train, y_train)        # ANN modeli eğitim verisi ile eğitilir
    prediction = model.predict(X_test) # Test verisi için tahmin yapılır

    y_true.append(y_test[0])            # Gerçek değer listeye eklenir
    y_pred.append(prediction[0])        # Tahmin edilen değer listeye eklenir

# =========================
# 6) Performans
# =========================
mae = mean_absolute_error(y_true, y_pred)                 # Ortalama mutlak hata (MAE) hesaplanır
rmse = np.sqrt(mean_squared_error(y_true, y_pred))        # Kök ortalama kare hata (RMSE) hesaplanır

print("ANN Sonuçları (LOO-CV)")       # Sonuç başlığı yazdırılır
print("----------------------")
print(f"MAE  = {mae:.4f}")            # MAE değeri ekrana yazdırılır
print(f"RMSE = {rmse:.4f}")           # RMSE değeri ekrana yazdırılır

results = pd.DataFrame({              # Gerçek ve tahmin değerlerini gösteren tablo oluşturulur
    "Gerçek_Zorluk": y_true,           # Gerçek zorluk değerleri
    "Tahmin_Zorluk": y_pred,           # ANN tarafından tahmin edilen zorluk değerleri
    "Hata": np.array(y_pred) - np.array(y_true)  # Tahmin hatası (tahmin - gerçek)
})

print("\nTahmin Tablosu:")             # Tablo başlığı yazdırılır
print(results.round(4))                # Sonuç tablosu 4 ondalık basamakla ekrana yazdırılır


ANN Sonuçları (LOO-CV)
----------------------
MAE  = 0.5623
RMSE = 0.6738

Tahmin Tablosu:
   Gerçek_Zorluk  Tahmin_Zorluk    Hata
0         2.2562         1.7812 -0.4750
1         4.9416         3.8085 -1.1332
2         6.0676         4.9034 -1.1642
3         4.1760         3.7960 -0.3800
4         1.0387         1.3314  0.2927
5         4.9916         5.4505  0.4589
6         1.0847         1.1078  0.0232
7         2.4409         3.0119  0.5710
